In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication
from codeflare_sdk.kubeflow.client.training_client import TrainingClient
from kubeflow.training.constants import constants

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "XXXXXXXX",
    server = "XXXXXXXXX",
    skip_tls=False
)
auth.login()

In [ ]:
tc = TrainingClient(job_kind=constants.TFJOB_KIND)

In [ ]:
from kubernetes.client import V1Container
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container
from kubeflow.training.api.training_client import TrainingClient
from kubeflow.training import KubeflowOrgV1ReplicaSpec, KubeflowOrgV1TFJobSpec,KubeflowOrgV1TFJob, KubeflowOrgV1RunPolicy

container = V1Container(
	name="tensorflow2",
	image="gcr.io/kubeflow-ci/tf-mnist-with-summaries:1.0",
	command=[
    	"python",
    	"/var/tf_mnist/mnist_with_summaries.py",
    	"--learning_rate=0.01",
    	"--batch_size=150"
    	]
)

worker = KubeflowOrgV1ReplicaSpec(
	replicas=1,
	restart_policy="Never",
	template=V1PodTemplateSpec(
    	spec=V1PodSpec(
        	containers=[container]
    	)
	)
)
run_policy=KubeflowOrgV1RunPolicy(
	active_deadline_seconds = None,
	backoff_limit = None,
	clean_pod_policy = None,
	scheduling_policy = None,
	suspend = False,
	ttl_seconds_after_finished = None
)
tfjob = KubeflowOrgV1TFJob(
	api_version="kubeflow.org/v1",
	kind="TFJob",
	metadata=V1ObjectMeta(name="mnist-examples",namespace="mark-dsp"),
	spec=KubeflowOrgV1TFJobSpec(
    	run_policy=run_policy,
    	tf_replica_specs={"Worker": worker}
	)
)

In [ ]:
tc.create_job(namespace="mark-dsp", job=tfjob)

In [ ]:
print(tc.list_jobs(namespace="mark-dsp", job_kind=constants.TFJOB_KIND))

In [ ]:
print(tc.get_job_conditions(name="mnist-examples", namespace="mark-dsp", job_kind=constants.TFJOB_KIND))

In [ ]:
tc.is_job_created(name="mnist-examples", namespace="mark-dsp", job_kind=constants.TFJOB_KIND)

In [ ]:
tc.is_job_succeeded(name="mnist-examples", namespace="mark-dsp", job_kind=constants.TFJOB_KIND)

In [ ]:
tc.is_job_restarting(name="mnist-examples", namespace="mark-dsp", job_kind=constants.TFJOB_KIND)

In [ ]:
tc.is_job_running(name="mnist-examples", namespace="mark-dsp", job_kind=constants.TFJOB_KIND)

In [ ]:
tc.is_job_failed(name="mnist-examples", namespace="mark-dsp", job_kind=constants.TFJOB_KIND)

In [ ]:
tfjob = tc.get_job(name="mnist-examples")

In [ ]:
tc.wait_for_job_conditions(name="mnist-examples")

In [ ]:
pods = tc.get_job_pods(name="mnist-examples")
print(pods)

In [ ]:
pod_names = tc.get_job_pod_names(name="mnist-examples")
print(pod_names)

In [ ]:
print(tc.get_job_logs(name="mnist-examples"))

In [ ]:
tc.update_job(tfjob, "mnist-examples")

In [ ]:
tc.delete_job(name="mnist-examples")